Some of the imports are not useful yet

In [110]:
from __future__ import print_function
import os
import numpy as np
import theano
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
from six.moves import range
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Dropout, Activation, Flatten, Masking
from keras.layers.convolutional import Convolution1D, MaxPooling1D, Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

In [34]:
import csv
dataset_xs = []
dataset_y = []
with open('data/ismis.csv', 'rb') as csvfile:
    next(csvfile) # skip header
    reader = csv.reader(csvfile)
    for row in reader:
        features_strings = row[1:147] + row[168:191] 
        dataset_xs.append(map(float, features_strings)) # features
        dataset_y.append(row[-1]) # genre
    
print(len(dataset_xs[0]))
print(type(dataset_xs[0][0]))
print(dataset_y[0])

classes = set(dataset_y)
nb_classes = len(classes)
mappings = dict(enumerate(classes))
mappings_rev = {v: k for k, v in mappings.items()}
print(mappings_rev)
dataset_y_mapped = map(lambda x: mappings_rev[x], dataset_y)
print(set(dataset_y_mapped))


169
<type 'float'>
Pop
{'Classical': 0, 'Jazz': 1, 'Metal': 2, 'Pop': 3, 'Rock': 4, 'Blues': 5}
set([0, 1, 2, 3, 4, 5])


In [36]:
def evens(dataset):
    return dataset[::2]
def odds(dataset):
    return dataset[1::2]

train_dataset_xs = odds(dataset_xs)
train_dataset_y = odds(dataset_y_mapped)
test_dataset_xs = evens(dataset_xs)
test_dataset_y = evens(dataset_y_mapped)

def randomize(dataset_xs, dataset_y):
    permutation = np.random.permutation(len(dataset_y))
    shuffled_dataset_xs = np.asarray(dataset_xs)[permutation]
    shuffled_dataset_y = np.asarray(dataset_y)[permutation]
    return shuffled_dataset_xs, shuffled_dataset_y

train_dataset_xs, train_dataset_y = randomize(train_dataset_xs,train_dataset_y)
test_dataset_xs, test_dataset_y = randomize(test_dataset_xs, test_dataset_y)

try:
    f = open('data/ismis_randomized.pickle', 'wb')
    save = {
        'train_dataset_xs': train_dataset_xs,
        'train_dataset_y': train_dataset_y,
        'test_dataset_xs': test_dataset_xs,
        'test_dataset_y': test_dataset_y,
        'mappings': mappings
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise

You can start here
=====

In [9]:
with open('data/ismis_randomized.pickle', 'rb') as f:
    save = pickle.load(f)
    train_dataset_xs = save['train_dataset_xs']
    train_dataset_y = save['train_dataset_y']
    test_dataset_xs = save['test_dataset_xs']
    test_dataset_y = save['test_dataset_y']
    del save  # gc


y_train = np_utils.to_categorical(train_dataset_y, nb_classes)
y_test = np_utils.to_categorical(test_dataset_y, nb_classes)

print(train_dataset_xs.shape)

(6247, 169)


Training & Testing:
=========================

In [14]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution1D
from keras.datasets import imdb
from keras import backend as K

# set parameters:
max_features = 5000
maxlen = 169
batch_size = 32
embedding_dims = 50
nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 2

In [ ]:

print(len(train_dataset_xs), 'train sequences')
print(len(test_dataset_xs), 'test sequences')

print('Pad sequences (samples x time)')
train_dataset_xs = sequence.pad_sequences(train_dataset_xs, maxlen=maxlen)
test_dataset_xs = sequence.pad_sequences(test_dataset_xs, maxlen=maxlen)
print('X_train shape:', train_dataset_xs.shape)
print('X_test shape:', test_dataset_xs.shape)

print('Build model...')
model = Sequential()

# # we start off with an efficient embedding layer which maps
# # our vocab indices into embedding_dims dimensions
# model.add(Embedding(max_features,
#                     embedding_dims,
#                     input_length=maxlen,
#                     dropout=0.2))

model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))

def max_1d(X):
    return K.max(X, axis=1)

model.add(Lambda(max_1d, output_shape=(nb_filter,)))

model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(6))
model.add(Activation('relu'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(train_dataset_xs, y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          validation_data=(test_dataset_xs, y_test))